### LeNet

In [31]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Activation, ZeroPadding2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.datasets import mnist, cifar10
from keras.utils import np_utils
import keras

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
img_rows = x_train[0].shape[0]
img_cols = x_train[0].shape[1]

In [4]:
# => 60000, 28, 28, 1
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

In [5]:
input_shape = (img_rows, img_cols, 1)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

x_train /= 255
x_test /= 255

In [6]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [12]:
num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2] # 28*28=784

In [15]:
# LeNet
model = Sequential()
model.add(Conv2D(20, (5,5), padding="same", input_shape=input_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(20, (5,5), padding="same", activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(500, activation="relu"))
model.add(Dense(num_classes, activation="softmax"))

In [16]:
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 20)        520       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 20)        10020     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 20)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 980)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               490500    
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [18]:
batch_size = 28
epochs = 10
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,validation_data=(x_test, y_test), shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 220s 4ms/step - loss: 0.1101 - accuracy: 0.9651 - val_loss: 0.0330 - val_accuracy: 0.9881
Epoch 2/10
60000/60000 [==============================] - 221s 4ms/step - loss: 0.0365 - accuracy: 0.9883 - val_loss: 0.0282 - val_accuracy: 0.9898
Epoch 3/10
60000/60000 [==============================] - 217s 4ms/step - loss: 0.0246 - accuracy: 0.9925 - val_loss: 0.0245 - val_accuracy: 0.9920
Epoch 4/10
60000/60000 [==============================] - 266s 4ms/step - loss: 0.0178 - accuracy: 0.9946 - val_loss: 0.0315 - val_accuracy: 0.9888
Epoch 5/10
60000/60000 [==============================] - 150s 3ms/step - loss: 0.0133 - accuracy: 0.9959 - val_loss: 0.0270 - val_accuracy: 0.9922
Epoch 6/10
60000/60000 [==============================] - 165s 3ms/step - loss: 0.0102 - accuracy: 0.9969 - val_loss: 0.0284 - val_accuracy: 0.9923
Epoch 7/10
600

In [19]:
model.save("LeNet.h5")

In [22]:
scores = model.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 8s 787us/step


In [24]:
print(f"accuracy: {scores[1]}")

accuracy: 0.9923999905586243


### AlexNet

In [27]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [37]:
l2_reg = 0
model = Sequential()
model.add(Conv2D(96, (11, 11), input_shape=x_train.shape[1:], padding="same", kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (5, 5), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(3072))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation("softmax"))

In [38]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 32, 32, 96)        34944     
_________________________________________________________________
batch_normalization_20 (Batc (None, 32, 32, 96)        384       
_________________________________________________________________
activation_17 (Activation)   (None, 32, 32, 96)        0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 16, 16, 256)       614656    
_________________________________________________________________
batch_normalization_21 (Batc (None, 16, 16, 256)       1024      
_________________________________________________________________
activation_18 (Activation)   (None, 16, 16, 256)     

In [40]:
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

In [41]:
batch_size = 32
epochs = 1

In [42]:
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
 8256/50000 [===>..........................] - ETA: 2:57:16 - loss: 1.8392 - accuracy: 0.3350

KeyboardInterrupt: 

In [ ]:
model.save("AlexNet.h5")

In [43]:
scor = model.evaluate(x_test, y_test)
print(f"score: {scor[1]}")

  224/10000 [..............................] - ETA: 18:03

KeyboardInterrupt: 